# CO2 Dashboard

A dashboard created to showcase holoviz panel using historic world $\text{CO}_2$ data.

the graphs are interactive within JupyterLab and can be ran as an interactive dashboard.

Thank you to thu-vu92 for the youtube video allowing for the understanding to build up upon design.
Her original dashboard can be found at https://github.com/thu-vu92/python-dashboard-panel 

Todo list:
    * Change barchart to geopandas display showing severity of $CO_2$ production for each country on a map.
    * Change scatterplot x_axis to a more readable value
    * remove index from table. Currently cannot be completed within panel system.

In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas
from IPython import get_ipython

In [2]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv('https://nyc3.digitaloceanspaces.com/owid-public/data/co2/owid-co2-data.csv')

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

In [3]:
''' replace underscore with space for presentation '''
df.columns = [_name.replace('_', ' ') for _name in df.columns]

# data preprocessing

There are 821119 missing values. Removing these rows or columns would lose a lot of important information therefore all missing values will be replaced with 0s and infer that if the value is 0 the data is missing. There are already 0s within the the dataframe, but as the data is to be used to find the countries with the highest C02 usage this is acceptable.   

In [4]:
# Fill NAs with 0s
df.fillna(0, inplace=True)

# Create GDP per capita column
df['gdp per capita'] = np.where(df['population']!=0, df['gdp']/df['population'],0)

# Creating the Dashboard

In [5]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

## Table of CO2 emission over time by country

In [6]:
''' Values within 'country' column to use for the displays '''
continents = ['World', 'Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antartica']
income = ['Lower-middle-income countries', 'Low-income countries', 'Upper-middle-income countries', 'High-income countries']
countries = list(set(df.country.unique().tolist())-set(continents)-set(income)-set(['North America (excl. USA)','EU-27', 'Kuwaiti Oil Fires',
                                                                                    'International transport', 'European Union (28)', 'European Union (27)',
                                                                                    'Europe (excl. EU-27)','Europe (excl. EU-28)','Asia (excl. China & India)']))
display_co2 = ['country','co2','coal co2',
       'flaring co2', 'gas co2', 'oil co2',
       'other industry co2',]
display_co2_per_capita = ['country','co2 per capita','coal co2 per capita',
       'flaring co2 per capita', 'gas co2 per capita', 'oil co2 per capita',
       'other co2 per capita']

In [7]:
''' Create button co2 v co2 per capita''' 
co2_button = pn.widgets.RadioButtonGroup(
    name = 'y_axis',
    options = ['co2', 'co2 per capita'],
    button_type='success'
)

''' Create Year Slider '''
year_slider = pn.widgets.IntSlider(value=1990, start=df.year.min(), end=df.year.max())

'''throttle updating of widget to when slider is released'''
pn.config.throttled = True 

In [10]:
''' Creates the first load of the dataframe. this is how it will look before any widget is activated. '''
co2_pane = pn.panel(df[display_co2].loc[(df['year'] == year_slider.value) & (df['country'].isin(countries))].sort_values(by='co2', ascending=False).head(10), sizing_mode='stretch_width')

In [11]:
''' Defines what will happen to the dataframe once a widget is used. Here the column_pane is transformed via the button value. 
    This is then fed in to the pane.object which transformers the columns based on the slider value. the result is then returned '''
def df_callback_slider(event):
    column_pane = df[display_co2].sort_values(by='co2', ascending=False) if co2_button.value == 'co2' else df[display_co2_per_capita].sort_values(by='co2 per capita', ascending=False)
    co2_pane.object = column_pane.loc[(df['year'] == year_slider.value) & (df['country'].isin(countries))].head(10)

In [12]:
'''set button and slider to update dataframe when changed'''
co2_button.param.watch(df_callback_slider, 'value')
year_slider.param.watch(df_callback_slider, 'value')

''' add button, slider and dataframe to column '''
top10_co2_table = pn.Column(co2_button, year_slider,  co2_pane,sizing_mode='stretch_width')

In [13]:
top10_co2_table

Column(sizing_mode='stretch_width')
    [0] RadioButtonGroup(button_type='success', name='y_axis', options=['co2', 'co2 per capita'], value='co2')
    [1] IntSlider(end=2020, start=1750, value=1990)
    [2] DataFrame(DataFrame, sizing_mode='stretch_width')

# Connect data pipeline to widgets

In [14]:
co2_pipeline = (
    idf[
        (idf.year <=year_slider) &
        (idf.country.isin(continents))
        
    ]
    .groupby(['country','year'])[co2_button].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year', ascending=True)
    .reset_index(drop=True)
)
                                 

In [15]:
co2_plot = co2_pipeline.hvplot(x = 'year', by = 'country', y = co2_button, line_width = 2, title = "CO2 emissions by continent")

co2_plot

## co2_vs_gdp_scatterplot_pipeline

In [54]:
co2_vs_gdp_scatterplot_pipeline = (
    idf[
        (idf.year == year_slider) &
         (idf.country.isin(countries))
    ]
    .groupby(['country', 'year', 'gdp'])['co2'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)


In [65]:
co2_vs_gdp_scatterplot = co2_vs_gdp_scatterplot_pipeline.hvplot(x='gdp', 
                                                                y='co2', 
                                                                by='country', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                min_height=200,
                                                                title = "Country CO2 v GDP "
                                                               )

In [66]:
co2_vs_gdp_scatterplot


## Bar chart with CO2 sources by continent


In [19]:
options = display_co2.copy()
options.remove('country')

In [20]:
yaxis_co2_source = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=options, 
    button_type='success'
)

In [26]:
continents_excl_world = ['Asia', 'Oceania', 'Europe', 'Africa', 'North America', 'South America', 'Antarctica']

co2_source_bar_pipeline = (
    idf[
        (idf.year == year_slider) &
        (idf.country.isin(continents_excl_world))
    ]
    .groupby(['year', 'country'])[yaxis_co2_source].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [27]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='country', 
                                                     y=yaxis_co2_source, 
                                                     color = 'green',
                                                     title='CO2 source by continent')

In [28]:
co2_source_bar_plot


# Creating Dashboard


In [31]:
#Layout using Template
template = pn.template.MaterialTemplate(
    title='World CO2 emission dashboard', 
    sidebar=[pn.pane.Markdown("# Country CO2 Emissions"), 
             pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change. It’s widely recognised that to avoid the worst impacts of climate change, the world needs to urgently reduce emissions. But, how this responsibility is shared between regions, countries, and individuals has been an endless point of contention in international discussions."), 
             
             pn.pane.Markdown("## Settings"),   
             co2_button, year_slider],
    main=[pn.Row(pn.Column( 
                           co2_plot.panel(width=600), margin=(0,25)),
                 co2_pane, height=400
                 ), 
          pn.Row(pn.Column(co2_vs_gdp_scatterplot.panel(width=500), margin=(0,25)), 
                 pn.Column(co2_source_bar_plot.panel(),yaxis_co2_source),sizing_mode='stretch_width'), 
         ],
)

template.servable();

### Run the dashboard
To run the dashboard write the following into the terminal  
`cd Documents/Data_Analysis/JupyterLab`  
`panel serve Dashboard.ipynb --show`

Then copy the url and paste into firefox

In [43]:
''' run to open dashboard in web browser '''
! panel serve CO2_Dashboard.ipynb --show

2022-05-25 15:19:34,079 Starting Bokeh server version 2.4.2 (running on Tornado 6.1)
2022-05-25 15:19:34,080 User authentication hooks NOT provided (default user enabled)
2022-05-25 15:19:34,082 Bokeh app running at: http://localhost:5006/CO2_Dashboard
2022-05-25 15:19:34,082 Starting Bokeh server with process id: 23827
2022-05-25 15:19:50,724 Error running application handler <bokeh.application.handlers.notebook.NotebookHandler object at 0x7fa2c755e340>: 'NoneType' object has no attribute 'system'
File 'CO2_Dashboard.ipynb', line 301, in <module>:
"        }) \n", Traceback (most recent call last):
  File "/home/ryan/anaconda3/lib/python3.9/site-packages/bokeh/application/handlers/code_runner.py", line 231, in run
    exec(self._code, module.__dict__)
  File "/home/ryan/Documents/Data_Analysis/JupyterLab/CO2_Dashboard.ipynb", line 301, in <module>
    "        }) \n",
AttributeError: 'NoneType' object has no attribute 'system'
 
2022-05-25 15:19:52,277 WebSocket connection opened
2022